<a href="https://colab.research.google.com/github/noambassat/SupremeCourtClassifier/blob/main/8_JONATAN_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score,make_scorer, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
# from imblearn.under_sampling import RandomUnderSampler


In [ ]:
file_path = "final_sample_df.xlsx"

In [ ]:
data = pd.read_excel(file_path)

# EDA


In [ ]:
data.columns

In [ ]:
data.columns

In [ ]:
data['רע"פ בקשה אחרת או דלמטה מיוחד'].value_counts()
data[(data['רע"פ בקשה אחרת או דלמטה מיוחד'] == 'בקשה אחרת') | (data['רע"פ בקשה אחרת או דלמטה מיוחד'] == 'דלמטה מיוחד')].shape[0]

מחיקה של מה שהוא בקשה אחרת או דלמטה מיוחד

In [ ]:
data = data[(data['רע"פ בקשה אחרת או דלמטה מיוחד'] != 'בקשה אחרת') & (data['רע"פ בקשה אחרת או דלמטה מיוחד'] != 'דלמטה מיוחד')]

In [ ]:
check_data = data[[
       'בקשה לרשות ערעור התקבלה?',
       'הערעור התקבל?']]

In [ ]:
check_data[check_data['בקשה לרשות ערעור התקבלה?'].isnull()]

In [ ]:
check_data[check_data[ 'הערעור התקבל?'].isnull()]

\אם הערך ריק ב"בקשה לרשות הערעור" - סימן שהתיק הזה לא נבדק.

In [ ]:
clean_data = data.dropna(subset = ['בקשה לרשות ערעור התקבלה?'])

In [ ]:
 clean_data['בקשה לרשות ערעור התקבלה?'].unique()

In [ ]:
clean_data["second_classifier_treshold_prediction"].unique()

אנחנו מתעלמים מכל מה שהבקשה שלו מסווגת כ"אחר"

In [ ]:
clean_data = data[(data['בקשה לרשות ערעור התקבלה?'] != 'אחר')]

In [ ]:
clean_data.shape

In [ ]:
clean_data = data.dropna(subset=["בקשה לרשות ערעור התקבלה?"])

In [ ]:
clean_data.shape

אנו מסווגים את כל מה שחלקי\התקבל כ1, ומה שנדחה כ0.

In [ ]:
clean_data['binary_outcome_for_first_classifier'] = clean_data['בקשה לרשות ערעור התקבלה?'].apply(lambda x: 0 if x == 'נדחה' else 1)

In [ ]:
clean_data['binary_outcome_for_first_classifier'].value_counts()

In [ ]:

def create_CM(data, labeled_col, predicted_col):
    # יצירת confusion matrix
    cm = confusion_matrix(data[labeled_col], data[predicted_col])

    # הצגת תוצאות confusion matrix
    print("Confusion Matrix:")
    print(cm)

    # ויזואליזציה של confusion matrix
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Class 0', 'Class 1'])
    disp.plot(cmap=plt.cm.Blues)
    plt.title("Confusion Matrix")
    if labeled_col.find("threshold") != -1:
        plt.title("Confusion Matrix for Threshold")
    plt.show()

    # חישוב Accuracy ו-F1 Score
    accuracy = accuracy_score(data[labeled_col], data[predicted_col])
    f1 = f1_score(data[labeled_col], data[predicted_col])

    # הצגת Accuracy ו-F1 Score
    print(f"Accuracy: {accuracy:.2f}")
    print(f"F1 Score: {f1:.2f}")



## First classifier

In [ ]:
create_CM(clean_data,"first_classifier_treshold_prediction",'binary_outcome_for_first_classifier')
# בדיקה עבור ערכים עם סף סיווג = 0.67

In [ ]:
create_CM(clean_data,"first_classifier_original_prediction",'binary_outcome_for_first_classifier')
# בדיקת הערכים ללא סף הסיווג (דיפולטי 0.5)

# מסווג שני

In [ ]:
[["מספר תיק", "מספר הליך","Year","מדינת ישראל בצד א'",
    'גוף המסמך חתוך', 'גוף המסמך חתוך למסווג השני',
       'גוף המסמך חתוך - embeddings',
       'גוף המסמך חתוך למסווג השני - embeddings',
       'first_classifier_predicted_probability',
       'first_classifier_original_prediction',
       'first_classifier_treshold_prediction',
       'second_classifier_predicted_probability',
       'second_classifier_original_prediction',
       'second_classifier_treshold_prediction']
]

In [ ]:
check_data = clean_data.dropna(subset=["הערעור התקבל?"])
check_data[check_data["second_classifier_treshold_prediction"].isna()]

In [ ]:
check_data['הערעור התקבל?'].unique()

In [ ]:
check_data[check_data['הערעור התקבל?']=="אחר"]

In [ ]:
check_data = check_data[check_data['הערעור התקבל?']!="אחר"]

In [ ]:
check_data['הערעור התקבל?'].unique()

In [ ]:
check_data['binary_outcome_for_second_classifier'] = clean_data['הערעור התקבל?'].apply(lambda x: 0 if x == 'נדחה' else 1)

In [ ]:
check_data.dropna(subset=["second_classifier_treshold_prediction"],inplace = True)

In [ ]:
create_CM(check_data,'second_classifier_treshold_prediction','binary_outcome_for_second_classifier')
# עבור סף סיווג של 0.9

In [ ]:
create_CM(check_data,"second_classifier_original_prediction",'binary_outcome_for_second_classifier')
# עבור סף סיווג דיפלוטי